In [11]:
import os
os.chdir('/Users/francopiccolo/GitHub/learn-to-rank')

In [12]:
import lightgbm as lgb

In [13]:
import pandas as pd
X_train = pd.read_pickle('./data/X_train.pkl')
X_val = pd.read_pickle('./data/X_val.pkl')

y_train = pd.read_pickle('./data/y_train.pkl')
y_val = pd.read_pickle('./data/y_val.pkl')

train_query_sizes = pd.read_pickle('./data/train_query_sizes.pkl')
val_query_sizes = pd.read_pickle('./data/val_query_sizes.pkl')

In [14]:
gbm = lgb.LGBMRanker(random_state=1)
gbm.fit(X_train, y_train, group=train_query_sizes,
        eval_set=[(X_val, y_val)], eval_group=[val_query_sizes], eval_at = [1, 3, 5, 10],
        early_stopping_rounds=10, verbose=True, 
        callbacks=[lgb.reset_parameter(learning_rate=lambda x: 0.95 ** x * 0.1)])

[1]	valid_0's ndcg@1: 0.242383	valid_0's ndcg@3: 0.386117	valid_0's ndcg@5: 0.454693	valid_0's ndcg@10: 0.527776
Training until validation scores don't improve for 10 rounds
[2]	valid_0's ndcg@1: 0.235163	valid_0's ndcg@3: 0.38034	valid_0's ndcg@5: 0.448688	valid_0's ndcg@10: 0.524324
[3]	valid_0's ndcg@1: 0.24012	valid_0's ndcg@3: 0.384039	valid_0's ndcg@5: 0.453747	valid_0's ndcg@10: 0.528783
[4]	valid_0's ndcg@1: 0.240884	valid_0's ndcg@3: 0.387748	valid_0's ndcg@5: 0.4567	valid_0's ndcg@10: 0.53069
[5]	valid_0's ndcg@1: 0.244896	valid_0's ndcg@3: 0.391115	valid_0's ndcg@5: 0.459497	valid_0's ndcg@10: 0.533142
[6]	valid_0's ndcg@1: 0.244902	valid_0's ndcg@3: 0.393382	valid_0's ndcg@5: 0.461242	valid_0's ndcg@10: 0.534467
[7]	valid_0's ndcg@1: 0.249738	valid_0's ndcg@3: 0.396487	valid_0's ndcg@5: 0.463993	valid_0's ndcg@10: 0.536602
[8]	valid_0's ndcg@1: 0.251731	valid_0's ndcg@3: 0.397885	valid_0's ndcg@5: 0.46568	valid_0's ndcg@10: 0.53809
[9]	valid_0's ndcg@1: 0.252109	valid_0's n

LGBMRanker(random_state=1)

In [15]:
pd.DataFrame(gbm.best_score_)

,valid_0
ndcg@1,0.264935
ndcg@10,0.550084
ndcg@3,0.414738
ndcg@5,0.481034


In [16]:
gbm.booster_.save_model('./data/model.txt')

In [17]:
# baseline model and param optimization
# wont go into it

In [18]:
# lightgbm api
# train_ds = lgb.Dataset('./data/train_ds.bin')
# val_ds = lgb.Dataset('./data/val_ds.bin')
# param = {'objective': 'lambdarank',
#          'metric': 'ndcg'}
# bst = lgb.train(param, train_ds, valid_sets=[val_ds], early_stopping_rounds=5)
# bst.save_model('model.txt', num_iteration=bst.best_iteration)
# bst = lgb.Booster(model_file='model.txt')
# ypred = bst.predict(test_ds) # num_iteration=bst.best_iteration